# AtCoder精進比較チャート

自分とライバルの精進量の比較ができます。 精進してライバルに差をつけましょう！

## 使い方

1. 「前処理」タブを実行する
1. 「設定」タブのUserIDとRaivalIDにIDを入力する。ライバルはカンマ区切りで複数入力できます
1. 「設定」タブを実行し、チャートを描画する

- チャートにカーソルを当てるとその日のサマリーが表示されます。

## 注意

※[kenkooooさんのAPI](https://github.com/kenkoooo/AtCoderProblems/blob/master/doc/api.md)を使わせていただいています。過度なリクエストは避け感謝しながら使いましょう。下記プログラムでは1リクエストにつき1秒のスリープを入れています。

※リクエスト制限のためライバルが多い場合や期間が長い場合にチャート表示に時間がかかる場合があります。

## 前処理

### ロガー

In [2]:
import logging
import sys

FORMAT = (
    "%(levelname)-8s %(asctime)s [%(filename)s %(funcName)s:%(lineno)s] %(message)s"
)


def get_logger(module_name: str, level=20, out_file: str = "") -> logging.Logger:
    """モジュールごとに設定可能なロガーを返す

    使用してるライブラリのログに興味がないため、ルートロガーの設定は行っていない

    level:
        10: DEBUG
        20: INFO
        30: WARNING
        40: ERROR
        50: CRITICAL

    Args:
        module_name (str): モジュール名
        level (int, optional): ログレベル. Defaults to 10.
        out_file (str, optional): ログ出力ファイル. Defaults to "".

    Returns:
        logging.Logger: ロガー
    """

    logger = logging.getLogger(module_name)
    logger.setLevel(level)
    logger.propagate = False

    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.DEBUG)
    handler.setFormatter(logging.Formatter(FORMAT))
    logger.addHandler(handler)

    if out_file:
        fl_handler = logging.FileHandler(filename=out_file, encoding="utf-8")
        fl_handler.setLevel(logging.DEBUG)
        fl_handler.setFormatter(logging.Formatter(FORMAT))
        logger.addHandler(fl_handler)

    return logger


### 実装

In [3]:
import datetime
import json
import time

import plotly.graph_objects as go
import requests


log = get_logger(__name__, level=20)

SUB_API_URL = "https://kenkoooo.com/atcoder/atcoder-api/v3/user/submissions?user={}&from_second={}"
MAX_SUB = 500


def get_user_submissions(user: str, epoch: int) -> dict:
    res = requests.get(SUB_API_URL.format(user, epoch))
    log.info(SUB_API_URL.format(user, epoch))
    subs = json.loads(res.text)
    if len(subs) < MAX_SUB:
        return subs

    time.sleep(1)
    max_epoch = max([sub["epoch_second"] for sub in subs])
    return subs + get_user_submissions(user, max_epoch)


def retrieve_unique_AC_subs(user: str, period: int):
    # get subs
    epoch = (datetime.datetime.now() + datetime.timedelta(days=-period)).timestamp()
    subs = get_user_submissions(user, int(epoch))

    # exclude AHC
    subs = [sub for sub in subs if not sub["contest_id"].lower().startswith("ahc")]
    subs = [
        sub for sub in subs if 0 <= sub["point"] <= 3000
    ]  # ヒューリスティックコンのリストでチェックするのが面倒なため3000点超えてたら除外
    log.debug(f"Algo subs {len(subs)}")

    # extract AC subs
    subs = [sub for sub in subs if sub["result"] == "AC"]
    log.debug(f"AC subs {len(subs)}")

    # add date
    for sub in subs:
        d = datetime.datetime.fromtimestamp(sub["epoch_second"])
        sub["date"] = d.strftime("%Y-%m-%d")

    # sort by date asc
    subs.sort(key=lambda x: x["date"])

    # extract unique AC
    unique_ac_subs = []
    unique_keys = set()
    for sub in subs:
        if sub["problem_id"] in unique_keys:
            continue
        unique_ac_subs.append(sub)
        unique_keys.add(sub["problem_id"])
    log.info(f"Unique AC subs {len(unique_ac_subs)}")

    return unique_ac_subs


def accumulate_y_score(subs: str, period: int, kind: str) -> list:
    date_to_score = {}
    for i in range(period + 1):
        d = datetime.datetime.now() + datetime.timedelta(days=-i)
        date = d.strftime("%Y-%m-%d")
        date_to_score[date] = 0
    for sub in subs:
        if kind == "獲得スコア":
            date_to_score[sub["date"]] += int(sub["point"])
        elif kind == "AC数":
            date_to_score[sub["date"]] += 1

    points_cum = [date_to_score[date] for date in sorted(date_to_score)]
    for i in range(1, len(points_cum)):
        points_cum[i] += points_cum[i - 1]

    return points_cum


def make_tooltip_text(dates: list, subs: list) -> list:
    day_summary = {}
    for date in dates:
        day_summary[date] = []

    for sub in subs:
        if sub["date"] not in day_summary:
            day_summary[date] = []
        problem = f'{sub["contest_id"]}  {sub["problem_id"].split("_")[-1]}  {int(sub["point"])}'
        day_summary[sub["date"]].append(problem)

    ret = []
    for date, daily_problems in sorted(day_summary.items()):
        daily_points = sum([int(x.split(" ")[-1]) for x in daily_problems])
        text = f"{date}<br>{len(daily_problems)} ACs,  {daily_points} Pts"
        if len(daily_problems) == 0:
            ret.append(text)
            continue

        daily_problems.sort()
        text += f"<br>{'- '*11}<br>" + "<br>".join(daily_problems)
        ret.append(text)

    return ret


def retrieve_chart_data(users: list, period: int, kind):
    dates = []
    for i in range(period + 1):
        d = datetime.datetime.now() + datetime.timedelta(days=-period + i)
        dates.append(d.strftime("%Y-%m-%d"))

    users_data = []
    for user in users:
        subs = retrieve_unique_AC_subs(user, period)
        x = dates
        y = accumulate_y_score(subs, period, kind)
        tooltip_text = make_tooltip_text(dates, subs)
        users_data.append((x, y, tooltip_text))

        time.sleep(1)
    return users_data


def draw_chart(users: list, period: int, kind="獲得スコア"):
    if kind not in ["AC数", "獲得スコア"]:
        log.critical("「獲得スコア」または「AC数」を指定してください")
        exit(1)

    users_data = retrieve_chart_data(users, period, kind)

    fig = go.Figure()

    # plot line chart
    for user, data in zip(users, users_data):
        fig.add_trace(
            go.Scatter(
                x=data[0],
                y=data[1],
                mode="lines",
                name=user,
                text=data[2],
                hovertemplate="%{text}",
            )
        )

    # set x axis date format
    fig.update_xaxes(
        dtick="M1" if period < 700 else "M6",
        tickformat="%Y-%m",
        tickangle=-45,
    )

    # layout
    fig.update_layout(
        title=f"精進量比較チャート （{kind}）",
        legend_title="Users",
        template="plotly",
        width=1200,
        height=700,
        font=dict(family="Courier New, monospace", size=18, color="Gray"),
    )

    # ツールチップの文字色を白に設定
    fig.update_traces(
        hoverlabel=dict(
            font=dict(color="white"),
        )
    )

    fig.show()


## チャートの描画

In [5]:
# @title 設定 { display-mode: "form" }

UserID = "chokudai" # @param {type:"string"}
RaivalID = "tourist, snuke" #@param {type:"string"}
Period = 700 #@param {type:"slider", min:10, max:365, step:5}
Mode = "獲得スコア" #@param ["獲得スコア", "AC数"]

users = [UserID] + RaivalID.replace(" ", "").split(",")

draw_chart(users, Period, kind=Mode)

INFO     2024-07-11 11:01:59,190 [4241687071.py get_user_submissions:17] https://kenkoooo.com/atcoder/atcoder-api/v3/user/submissions?user=chokudai&from_second=1660183319
INFO     2024-07-11 11:01:59,191 [4241687071.py retrieve_unique_AC_subs:59] Unique AC subs 31
INFO     2024-07-11 11:02:00,344 [4241687071.py get_user_submissions:17] https://kenkoooo.com/atcoder/atcoder-api/v3/user/submissions?user=tourist&from_second=1660183320
INFO     2024-07-11 11:02:00,346 [4241687071.py retrieve_unique_AC_subs:59] Unique AC subs 175
INFO     2024-07-11 11:02:01,584 [4241687071.py get_user_submissions:17] https://kenkoooo.com/atcoder/atcoder-api/v3/user/submissions?user=snuke&from_second=1660183321
INFO     2024-07-11 11:02:02,748 [4241687071.py get_user_submissions:17] https://kenkoooo.com/atcoder/atcoder-api/v3/user/submissions?user=snuke&from_second=1674479402
INFO     2024-07-11 11:02:03,979 [4241687071.py get_user_submissions:17] https://kenkoooo.com/atcoder/atcoder-api/v3/user/submissions?